In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf

In [ ]:
def getCrosstalk(filename):
    # Subroutine to retrieve the crosstalk results measured by
    # Adam Snyder at SLAC and put them in the form needed for ISR
    try:
        hdulist = pf.open(filename, mode='readonly', do_not_scale_image_data=True)
        ctData=hdulist[0].data
        assert ctData.shape == (16,16)
    except FileNotFoundError:
        print("Crosstalk file not found")
        sys.exit()
    except AssertionError:
        print("Crosstalk data not of the proper type")
        sys.exit()
    except:
        print("Failed to get crosstalk data")
        sys.exit()
    # Need to zero out the diagonal elements due to difference in convention
    for i in range(16):
        ctData[i,i] = 0.0
    return ctData

In [ ]:
ctSnyder = getCrosstalk('/Users/cslage/Research/LSST/code/auxtel/auxtel_crosstalk_matrix.fits')
ctDM = getCrosstalk('/Users/cslage/Research/LSST/code/auxtel/auxtel_crosstalk_matrix_DM.fits')

In [ ]:
np.fill_diagonal(ctSnyder, 1.0)
diff = ((ctDM - ctSnyder) / ctSnyder) * 100.0
np.fill_diagonal(diff, 0)
for i in range(16):
    for j in range(16):
        if i == j:
            continue
        print(i,j,ctSnyder[i,j], ctDM[i,j], diff[i,j])

In [ ]:
np.fill_diagonal(ctSnyder, 1.0)
diff = abs((ctSnyder - ctDM) / ctSnyder)
np.fill_diagonal(diff, 0)

In [ ]:
plt.imshow(diff)

In [ ]:
plotted = np.zeros([16,16], dtype='int')
plt.figure(figsize=(16,12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Auxtel Crosstalk comparison - 01Mar21", fontsize=24)
for sep in range(1,4):
    if sep == 1:
        ymin = -2E-3; ymax = 2E-3
    else:
        ymin = -2E-4; ymax = 2E-4
    vals = []
    DMvals = []
    names = []
    for i in range(16):
        iHalf = i%8
        for j in range(16):
            jHalf = j%8
            AlreadyPlotted = False
            if abs(iHalf - jHalf) == sep and abs(i - j) == sep:
                names.append(f'A{i}V{j}')
                vals.append(ctSnyder[i,j])
                DMvals.append(ctDM[i,j])
                plotted[i,j] = 1
    plt.subplot(4,1,sep)
    plt.title(f'Separation = {sep} amps')
    plt.ylim(ymin, ymax)
    plt.plot(names, vals, label='Snyder')
    plt.plot(names, DMvals, label='DM')
    plt.xticks(names, rotation=90, fontsize=10)
    plt.legend()
for i in range(16):
    for j in range(16):
        if (plotted[i,j] == 0) and (i != j):
            names.append(f'A{i}V{j}')
            vals.append(ctSnyder[i,j])
            DMvals.append(ctDM[i,j])
plt.subplot(4,1,4)
plt.title('Separation = rest')
plt.ylim(ymin, ymax)
plt.plot(names, vals, label='Snyder')
plt.plot(names, DMvals, label='DM')
plt.xticks(names, rotation=90, fontsize=6)
plt.legend()
plt.savefig("/Users/cslage/Research/LSST/code/auxtel/Crosstalk_Comparison_01Mar21.pdf")